In [ ]:
############## Parse results from Brat Output files

In [1]:
import csv, requests, json, collections, os, pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import re
import numpy as np
from numpy.random import randint
regex = re.compile('[^a-zA-Z0-9-_]')
import xlwt 
from xlwt import Workbook
import pandas as pd
from os import listdir
from os.path import isfile, join
# from nltk.tokenize import word_tokenize
# from nltk import pos_tag
import stanza
nlp = stanza.Pipeline('en',verbose=False, processors='tokenize,pos,lemma')

In [2]:
######## Configuring the properties and entity categories
category_properties = {}
category_properties['Hardware-Devices'] = ['notRelevant','isPowered','isConnected','isSetup','isUsed']
category_properties['Software-Device-Drivers'] = ['notRelevant','isInstalled','isRelatedDeviceConnected','isSettingsChanged']
category_properties['Software-OS-Related'] = ['notRelevant','isOpened','isSettingsChanged']
category_properties['Software-Other'] = ['notRelevant','isInstalled','isOpened','isSettingsChanged']
category_properties['Hardware-Other'] = ['']

property_event = {}
property_event['notRelevant'] = True
property_event['isPowered'] = False
property_event['isConnected'] = False
property_event['isSetup'] = True
property_event['isUsed'] = True
property_event['isInstalled'] = False
property_event['isRelatedDeviceConnected'] = False
property_event['isSettingsChanged'] = True
property_event['isOpened'] = False

property_values = {}
property_values[True] = ['true','false']
property_values[False] = ['f->t','t->f','noChange','start-T','start-F']

In [3]:
step_details = []
mentions = {}
entities = {}
ent_variants = {}
ent_verb_spans = {}
merged_ents = {}
step_wise_properties = {}
ent_wise_spans = {}

all_annotations = pickle.load(open('./annotations/all_files_v1_complete.pkl','rb'))

In [4]:
################ A wide variety of util functions

# def process_word(word):
#     w = word
#     cap = False
#     if(w.isupper()):
#         cap = True
#     w = w.lower()
# #     print(w)
#     if('\'s' == w):
#         w=''
#     if('cd' == w):
#         w='CD'
#     w = lemmatizer.lemmatize(w)
# #     print(w)
#     if(not cap):
#         w = w.capitalize()
#     else:
#         w = w.upper()
#     w = regex.sub('', w)
#     if(len(w)<2 and w!= 'X'):
#         w=''
#     return w
##### Get line number for an entity
def get_line(offset):
    for i, step in enumerate(step_details):
        if(int(offset)<step[2]):
            return i-2
    return len(step_details)-1

##### Clean up the text by lemmatizing
def clean_text(ent_text):
    tokens=nlp(ent_text)
    tokens = [w for sent in tokens.sentences for w in sent.words]
    lemmas = [token.lemma.lower().strip() for token in tokens]
    ent_text = ' '.join(lemmas)
    return ent_text

##### Get base text for a mention ID
def get_text(mention):
    text = mentions[mention][-2]
    while(entities[text][0] != '' and entities[text][0] != text):
        text = entities[text][0]
    return text
##### get base text for a given entity form: example printers -> printer
def get_base_ent(text):
    while(entities[text][0] != '' and entities[text][0] != text):
        text = entities[text][0]
    return text
def state_change(new_state, prev_state):
    if(new_state==prev_state):
        return 'None'
    elif(new_state=='True'):
        return 'True'
    elif(new_state=='False'):
        return 'False'
    else:
        return 'None'
####### Get indexes for noun and verb positions
def get_indexes(sent_tokens, ent_text,verb=False):
    indexes = ['0' for _ in sent_tokens]
    if verb:
        indexes = ['1' if token.pos=='VERB' else '0' for token in sent_tokens]
    else:
        for ent in ent_variants[ent_text]:
            ent_tokens = nlp(ent)
            ent_tokens = [w for sent in ent_tokens.sentences for w in sent.words]
            for i in range(len(sent_tokens)-len(ent_tokens)+1):
                if(all(ent_tokens[j].lemma.lower().strip()==sent_tokens[i+j].lemma.lower().strip() for j in range(len(ent_tokens)))):
                    for j in range(len(ent_tokens)):
                        indexes[i+j]='1'
    return indexes
def process_sent(sent:str,ent_text:str):
    tokens=nlp(sent)
    tokens = [w for sent in tokens.sentences for w in sent.words]
    lemmas = [token.lemma.lower().strip() for token in tokens]
    ent_spans = get_indexes(tokens,ent_text)
    verb_spans = get_indexes(tokens, ent_text, verb=True)
    return [lemmas,ent_spans,verb_spans]

In [5]:
######## Parse annotations from a file and store it in state-change-type dataset format
######## Ignore the file in case of any errors whatsoever
def parse_annotations(out_file,doc_id=1,folder='config/',file='monitor',filter_prop='',model='bert',test_files={}):
    global step_details, mentions,entities,ent_variants,ent_verb_spans,merged_ents,step_wise_properties,ent_wise_spans
    try:
        steps = all_annotations[file]['steps']
        step_details = all_annotations[file]['step_details']
        mentions = all_annotations[file]['mentions']
        entities = all_annotations[file]['entities']
        ent_variants = all_annotations[file]['ent_variants']
        ent_verb_spans = all_annotations[file]['ent_verb_spans']
        merged_ents = all_annotations[file]['merged_ents']
        step_wise_properties = all_annotations[file]['step_wise_properties']
        for ent in step_wise_properties:
            for prop in step_wise_properties[ent]:
                if(filter_prop != '' and prop!=filter_prop):
                    continue
                for step_no in range(len(steps)-1):
                    if(model=='bert'):
                        query=''
                        ######## Convert query to natural text
                        if(prop=='isOpened'):
                            query = 'Is '+ ent +' opened?'
                        elif(prop=='isSettingsChanged'):
                            query = 'Were settings of ' + ent + ' changed?'
                        elif(prop=='isInstalled'):
                            query = 'Is ' + ent + ' installed?'
                        elif(prop=='isRelatedDeviceConnected'):
                            query = 'Is the device for ' + ent + ' connected?'
                        elif(prop=='isConnected'):
                            query = 'Is ' + ent + ' connected?'
                        elif(prop=='isSetup'):
                            query = 'Was ' + ent + ' setup?'
                        elif(prop=='isPowered'):
                            query = 'Is ' + ent + ' powered on?'
                        elif(prop=='isUsed'):
                            query = 'Was ' + ent + ' used?'
                        elif(prop=='notRelevant'):
                            continue
                        else:
                            print('ERROR '+prop)
                        if(property_event[prop]):
                            print(repr(query),repr(steps[step_no+1]),step_wise_properties[ent][prop][step_no+1],sep='\t\t\t',file=out_file)
                        else:
                            print(repr(query),repr(steps[step_no+1]),state_change(step_wise_properties[ent][prop][step_no+1],step_wise_properties[ent][prop][step_no]),sep='\t\t\t',file=out_file)
                        if(prop in test_files):
                            if(property_event[prop]):
                                print(repr(query),repr(steps[step_no+1]),step_wise_properties[ent][prop][step_no+1],sep='\t\t\t',file=test_files[prop])
                            else:
                                print(repr(query),repr(steps[step_no+1]),state_change(step_wise_properties[ent][prop][step_no+1],step_wise_properties[ent][prop][step_no]),sep='\t\t\t',file=test_files[prop])
                    elif(model=='prolocal'):
                        sent_lemmas, ent_spans, verb_spans = ent_verb_spans[ent][step_no]
                        if(property_event[prop]):
                            print('####'.join(sent_lemmas),','.join(verb_spans),','.join(ent_spans),step_wise_properties[ent][prop][step_no+1],sep='\t',file=out_file)
                        else:
                            print('####'.join(sent_lemmas),','.join(verb_spans),','.join(ent_spans),state_change(step_wise_properties[ent][prop][step_no+1],step_wise_properties[ent][prop][step_no]),sep='\t',file=out_file)
                    else:
                        print(model)
    except Exception as e:
        print(e)

In [6]:
# parse_annotations()
# f.close()

In [18]:
########## Parse all files for a given property. Pass empty property to take all props
def parse_prop(filter_prop='isOpened'):
    np.random.seed(1234)
    print('\n\n',filter_prop)
    train_f = open('./annotations/bert_annotations_'+filter_prop+'_train_comp.tsv','w',encoding='utf-8')
    dev_f = open('./annotations/bert_annotations_'+filter_prop+'_dev_comp.tsv','w',encoding='utf-8')
    test_f = open('./annotations/bert_annotations_'+filter_prop+'_test_comp.tsv','w',encoding='utf-8')
    data_folder = './annotations/'
    files = [train_f,dev_f,test_f]
    count = 1
    for folder in [fol for fol in listdir(data_folder) if not isfile(join(data_folder, fol))]:
        folder = data_folder + folder+'/'
        onlyfiles = [f[:-4] for f in listdir(folder) if f.endswith('.txt')]
        for file in onlyfiles:
            out_file = np.random.choice(files,p=[0.8,0.1,0.1])
#             print(count, folder, file,out_file.name[-10:])
            parse_annotations(out_file=out_file,doc_id=count, folder=folder, file=file, filter_prop=filter_prop)
            count+=1
    train_f.close()
    dev_f.close()
    test_f.close()

In [19]:
# parse_prop()

In [20]:
######## PArse for all props
for prop in property_event:
    if(prop.startswith('is')):
        parse_prop(prop)



 isPowered
'654_2___Set_up_an_FTP_Server_in_Ubuntu_Linux___Configuring_the_FTP_Server'
'278_1___Change_Process_Priorities_in_Windows_Task_Manager___Steps'


 isConnected
'654_2___Set_up_an_FTP_Server_in_Ubuntu_Linux___Configuring_the_FTP_Server'
'278_1___Change_Process_Priorities_in_Windows_Task_Manager___Steps'


 isSetup
'654_2___Set_up_an_FTP_Server_in_Ubuntu_Linux___Configuring_the_FTP_Server'
'278_1___Change_Process_Priorities_in_Windows_Task_Manager___Steps'


 isUsed
'654_2___Set_up_an_FTP_Server_in_Ubuntu_Linux___Configuring_the_FTP_Server'
'278_1___Change_Process_Priorities_in_Windows_Task_Manager___Steps'


 isInstalled
'654_2___Set_up_an_FTP_Server_in_Ubuntu_Linux___Configuring_the_FTP_Server'
'278_1___Change_Process_Priorities_in_Windows_Task_Manager___Steps'


 isRelatedDeviceConnected
'654_2___Set_up_an_FTP_Server_in_Ubuntu_Linux___Configuring_the_FTP_Server'
'278_1___Change_Process_Priorities_in_Windows_Task_Manager___Steps'


 isSettingsChanged
'654_2___Set_up_an_FTP_

In [10]:
# parse_annotations(open('temp.txt','w',encoding='utf-8'),169,'./annotations/Mac/','1017_1___Customize_Your_Mac_Using_the_Terminal___Using_Terminal_Commands','isOpened')

In [11]:
# parse_prop('')

In [12]:
####### Parse the dataste for all folders for a given property
def parse_prop_folders(filter_prop='isOpened'):
    np.random.seed(1234)
    print('\n\n',filter_prop)
    train_f = open('./annotations/bert_annotations_'+filter_prop+'_train.tsv','w',encoding='utf-8')
    dev_f = open('./annotations/bert_annotations_'+filter_prop+'_dev.tsv','w',encoding='utf-8')
    
    data_folder = './annotations/'
    count = 1
    for folder in [fol for fol in listdir(data_folder) if not isfile(join(data_folder, fol))]:
        test_f = open('./annotations/bert_annotations_'+filter_prop+'_test_'+folder+'.tsv','w',encoding='utf-8')
        files = [train_f,dev_f,test_f]
        folder = data_folder + folder+'/'
        onlyfiles = [f[:-4] for f in listdir(folder) if f.endswith('.txt')]
        for file in onlyfiles:
            out_file = np.random.choice(files,p=[0.8,0.1,0.1])
            print(count, folder, file,out_file.name[-10:])
            parse_annotations(out_file=out_file,doc_id=count, folder=folder, file=file, filter_prop=filter_prop)
            count+=1
        test_f.close()
    train_f.close()
    dev_f.close()
    

In [13]:
# parse_prop_folders()

In [14]:
####### Prepare the dataset for ProLocal model
def parse_prop_prolocal(filter_prop='isOpened'):
    np.random.seed(1234)
    print('\n\n',filter_prop)
    train_f = open('./annotations/prolocal_annotations_'+filter_prop+'_train.tsv','w',encoding='utf-8')
    dev_f = open('./annotations/prolocal_annotations_'+filter_prop+'_dev.tsv','w',encoding='utf-8')
    test_f = open('./annotations/prolocal_annotations_'+filter_prop+'_test.tsv','w',encoding='utf-8')
    data_folder = './annotations/'
    files = [train_f,dev_f,test_f]
    count = 1
    for folder in [fol for fol in listdir(data_folder) if not isfile(join(data_folder, fol))]:
        folder = data_folder + folder+'/'
        onlyfiles = [f[:-4] for f in listdir(folder) if f.endswith('.txt')]
        for file in onlyfiles:
            out_file = np.random.choice(files,p=[0.8,0.1,0.1])
            print(count, folder, file,out_file.name[-10:])
            parse_annotations(out_file=out_file,doc_id=count, folder=folder, file=file, filter_prop=filter_prop,model='prolocal')
            count+=1
    train_f.close()
    dev_f.close()
    test_f.close()

In [15]:
# parse_prop_prolocal()

In [16]:
###### Parse the dataset for common Bert model for all properties
def parse_prop_common(filter_prop=''):
    np.random.seed(1234)
    print('\n\n',filter_prop)
    train_f = open('./annotations/bert_annotations_'+filter_prop+'_train.tsv','w',encoding='utf-8')
    dev_f = open('./annotations/bert_annotations_'+filter_prop+'_dev.tsv','w',encoding='utf-8')
    test_f = open('./annotations/bert_annotations_'+filter_prop+'_test.tsv','w',encoding='utf-8')
    test_files={}
    for prop in property_event:
        test_files[prop] = open('./annotations/bert_annotations_'+filter_prop+'_test_'+prop+'.tsv','w',encoding='utf-8')
    data_folder = './annotations/'
    files = [train_f,dev_f,test_f]
    count = 1
    for folder in [fol for fol in listdir(data_folder) if not isfile(join(data_folder, fol))]:
        folder = data_folder + folder+'/'
        onlyfiles = [f[:-4] for f in listdir(folder) if f.endswith('.txt')]
        for file in onlyfiles:
            out_file = np.random.choice(files,p=[0.8,0.1,0.1])
            print(count, folder, file,out_file.name[-10:])
            if(out_file==files[-1]):
                parse_annotations(out_file=out_file,doc_id=count, folder=folder, file=file, filter_prop=filter_prop,test_files=test_files)
            else:
                parse_annotations(out_file=out_file,doc_id=count, folder=folder, file=file, filter_prop=filter_prop)
            count+=1
    train_f.close()
    dev_f.close()
    test_f.close()
    for prop in property_event:
        test_files[prop].close()

In [17]:
# parse_prop_common()